In [ ]:
import cv2
import numpy as np
import string
import datetime
import collections

In [ ]:
debug = False
from time import sleep

In [ ]:
def shutdown(cam):
    cv2.destroyAllWindows()
    cam.release()

In [ ]:
def extractRect(img, x, y, w, h):
    if img is None:
        return None
    
    return img[y:y+h,x:x+w]

In [ ]:
def capture(cam):
    raw = cam.read()[1]
    
    if raw is None:
        return None
    
    rightHalf = extractRect(raw, 320, 0, 290, 480)
    topHalf = extractRect(raw, 0, 0, 640, 240)
    
    gray = cv2.cvtColor(raw, cv2.COLOR_RGB2GRAY)
    
    return gray

In [ ]:
def drawRect(img, x, y, w, h):
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,0),1)

In [ ]:
def threshold(img):
    retval, threshold = cv2.threshold(img, 150, 255, cv2.THRESH_OTSU)
    
    return threshold

In [ ]:
def allEqual(aList):
    return all(aList[0] == item for item in aList)

In [ ]:
class Zone:
    
    def __init__(self, x, y, size):
        self.__x = x
        self.__y = y
        self.__w = size
        self.__h = size
        self.__hot = None
    
    def x(self):
        return self.__x
    
    def y(self):
        return self.__y
    
    def w(self):
        return self.__w
    
    def h(self):
        return self.__h
    
    def isHot(self):
        return self.__hot
    
    def update(self, data):
        zoneRect = extractRect(data, self.x(), self.y(), self.w(), self.h()) 
        self.__hot = isLighterThan50percentGrey(zoneRect)

In [ ]:
def isLighterThan50percentGrey(blackAndWhiteImg):
    return cv2.mean(blackAndWhiteImg)[0] < 128.0

In [ ]:
class Viewer:
        
    def render(self, rawFrame, filteredFrame, meter):
        key = cv2.waitKey(10)
        
        rawCopy = rawFrame.copy()
        filterCopy = filteredFrame.copy()
        
        for zone in meter.getZones():
            
            drawRect(rawCopy, zone.x(), zone.y(), zone.w(), zone.h())
            if zone.isHot():
                drawRect(filterCopy, zone.x(), zone.y(), zone.w(), zone.h())
        
        cv2.imshow('raw', rawCopy)
        cv2.imshow('filtered', filterCopy)

class NullViewer:
    def render(self, rawFrame, filteredFrame, meter):
        return

In [ ]:
class Monitor:
    
    def __init__(self, camera, meter, viewer = NullViewer()):
        self.__camera = camera
        self.__meter = meter
        self.__online = True
        self.__viewer = viewer
    
    def poll(self):
        newFrame = capture(self.__camera)
        self.__online = newFrame is not None
        
        if self.__online:
            filteredFrame = self.filterFrame(newFrame)
            flowQty = self.__meter.update(newFrame)
            
            self.__viewer.render(newFrame, filteredFrame, self.__meter)
            
            return flowQty
        else:
            raise Exception("camera offline!")
    
    def isOnline(self):
        return self.__online
    
    def filterFrame(self, rawFrame):
        return threshold(rawFrame)
    

In [ ]:
class Trigger:
    
    def __init__(self, zone1, zone2):
        self.__zone1 = zone1
        self.__zone2 = zone2
        self.__lastState = [None, None]
        self.__state = [None, None]
        
        self.__validStates = collections.deque(maxlen=4)
        self.__validStates.append([True,True])
        self.__validStates.append([False,True])
        self.__validStates.append([False,False])
        self.__validStates.append([True,False])
    
    def setNumber(self, num):
        self.__num = num
        
    def zones(self):
        return [self.__zone1, self.__zone2]
        
    def update(self, data):
        self.__zone1.update(data)
        self.__zone2.update(data)
        
        self.__lastState = list(self.__state)
        self.__state = [self.__zone1.isHot(), self.__zone2.isHot()]
        
        if not self.__knownState():
            while self.__validStates[0] != self.__state:
                self.__validStates.rotate(-1)
    
    def fired(self):
        if self.__hasChanged() and self.__knownState():     
            
            self.__validStates.rotate(-1)
            if self.__validStates[0] == self.__state:
                if allEqual(self.__state):
                    if debug:
                        print(self.__num, " : ", self.__lastState, " -> ", self.__state)
                    return True
            else:
                raise Exception('error on trigger', self.__num)
        return False
            
    
    def __hasChanged(self):
        return set(self.__lastState) != set(self.__state)
    
    def __knownState(self):
        return None not in self.__lastState
    
  

In [ ]:
class Meter:
    
    def __init__(self, name, triggers, sensitivity):
        self.__triggers = triggers
        self.__zones = []
        self.__name = name
        self.__lastFired = None
        self.__sensitivity = sensitivity
        self.__fireDeque = collections.deque(maxlen=len(triggers))
        
        trigCount = 0
        for item in triggers:
            self.__fireDeque.append(item)
            self.__zones.extend(item.zones())
            item.setNumber(trigCount)
            trigCount = trigCount + 1
        
    def update(self, data):
        fired = []
        for trigger in self.__triggers:
            trigger.update(data)
            if trigger.fired():
                fired.append(trigger)
            
        if len(fired) > 1:
            raise Exception("Two triggers fired together?")
        
        if len(fired) == 1:
            if self.__lastFired is None:
                while self.__fireDeque[0] is not fired[0]:
                    self.__fireDeque.rotate(-1)
                self.__fireDeque.rotate(1)
            
            self.__lastFired = fired[0]
            self.__fireDeque.rotate(-1)
            
            if self.__fireDeque[0] is not self.__lastFired:
                raise Exception("Unexpected trigger fired!")
            else:
                return self.__sensitivity
        
        return 0
            
        
    def getZones(self):
        return self.__zones

In [ ]:
cam = cv2.VideoCapture("file:///Users/jackhig/Desktop/meter_qt.mp4")
# cam = cv2.VideoCapture(0)

one = Trigger(Zone(40,240,20), Zone(40,220,20))
two = Trigger(Zone(60,160,20), Zone(60,140,20))
three = Trigger(Zone(90,80,20), Zone(110,60,20))
four = Trigger(Zone(150,30,20), Zone(170,10,20))
five = Trigger(Zone(230,0,20), Zone(250,0,20))
six = Trigger(Zone(285,0,40), Zone(325,0,40))
seven = Trigger(Zone(380,0,20), Zone(400,0,20))
eight = Trigger(Zone(460,10,20), Zone(480,30,20))
nine = Trigger(Zone(520,70,20), Zone(540,90,20))
ten = Trigger(Zone(560,130,20), Zone(570,150,20))

hotMeter = Meter("hot", [one, two, three, four, five, six, seven, eight, nine, ten], 0.1)

monitor = Monitor(cam, hotMeter)

while(monitor.isOnline()):
    flowQty = monitor.poll()
    
    if flowQty>0:
        print(flowQty)



In [ ]:
cam = cv2.VideoCapture("file:///Users/jackhig/Desktop/meter_qt.mp4")
# cam = cv2.VideoCapture(0)

triggerZoneSize = 20
triggerZones = []
triggerStates = []
fired = -1

triggerQty = 0.5
total = 0.0

def addTrigger(x, y):
    triggerZones.append((x, y, triggerZoneSize))
    triggerStates.append(-1)
    
# addTrigger(250, 190)
# addTrigger(260, 230)
# addTrigger(250, 270)

alphabet = string.ascii_lowercase

while True:
    liveImg = capture(cam)

    if liveImg is None:
        shutdown(cam)
        break
    
    thresholded = threshold(liveImg)
    
    thresholded_annotated = None
    if debug:
        cv2.imshow('thresh', thresholded)
        thresholded_annotated = threshold(liveImg)
    
    zoneCount = 0
    
    for zone in triggerZones:
        rect = extractRect(thresholded, zone[0], zone[1], zone[2], zone[2])
        drawRect(liveImg, zone[0], zone[1], zone[2], zone[2])
        
        if debug:
            drawRect(thresholded_annotated, zone[0], zone[1], zone[2], zone[2])
            
        blackOrWhite = 0
        if cv2.mean(rect)[0] > 128.0:
            blackOrWhite = 1
        else:
            blackOrWhite = 0
            
        triggerStates[zoneCount] = blackOrWhite
        
        if debug:
            cv2.imshow(alphabet[zoneCount], rect)
        zoneCount = zoneCount+1
    
    if len(triggerStates) > 0 and allEqual(triggerStates):
        if triggerStates[-1] is not fired and triggerStates[0] is not fired:
            if fired is -1:
                print('starting')
            else:
                now = datetime.datetime.now()
                total = total + triggerQty
                print(now, ":", total)
            fired = triggerStates[-1]
            
    
    if debug:
        cv2.imshow( 'live', liveImg )
        cv2.imshow( 'thresh', thresholded_annotated )


    key = cv2.waitKey(10)
    if key == 27:
        shutdown(cam)
        break

print ("Done.")